In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.losses import categorical_crossentropy

from tensorflow.keras.utils import to_categorical

from sklearn.metrics import f1_score

from sklearn.model_selection import cross_val_score

from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import cv2



from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.utils import class_weight
from tensorflow.keras.datasets import mnist

import statistics
import time
from bayes_opt import BayesianOptimization

Using TensorFlow backend.


In [2]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [3]:
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap=plt.cm.Blues) #'viridis'
    #plt.savefig('/home/jovyan/conf_matrix.png')
    plt.show()
    
    return conf_matx

In [4]:
def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(15,5), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    ax = fig.add_subplot(122)
    ax.plot(model_history.history['accuracy'])
    ax.plot(model_history.history['val_accuracy'])
    ax.set(title=model_name + ': Model Accuracy; test='+ str(np.round(model_history.history['val_accuracy'][-1], 3)),
           ylabel='Accuracy', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    #plt.savefig('/home/jovyan/curve.png')
    plt.show()
    
    return fig

In [5]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [6]:
width, height, channels = X_train.shape[1], X_train.shape[2], 1
X_train = X_train.reshape((X_train.shape[0], width, height, channels))
X_test = X_test.reshape((X_test.shape[0], width, height, channels))
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [7]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [8]:
y_test = to_categorical(Y_test)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=999 )
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(Y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(45, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(90, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(180, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
opt = keras.optimizers.Adam(lr=0.00425)

model.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=5,validation_data = (x_val, y_val), batch_size=64, verbose = 1)

In [ ]:
plot_history(history, 'first')

In [ ]:
_,acc = model.evaluate(X_test, y_test, verbose=0)
print(acc)

In [ ]:
test_preds = model.predict(X_test, batch_size=64)

In [ ]:
preds_df = pd.DataFrame(test_preds)

In [ ]:
predicted_labels = preds_df.idxmax(axis=1)
draw_confusion_matrix(Y_test, predicted_labels)

In [ ]:
print(classification_report(Y_test, predicted_labels))

### Bayesian optimization 

In [9]:
def generate_model(dropout, neuronPct, l_num):
    

    if l_num >= 0 and l_num <= 0.3 :
        l = 1
    if l_num >= 0.31 and l_num <= 0.6: 
        l = 2
    if l_num >= 0.61 and l_num <= 1: 
        l = 3

    neuronConunt = int(neuronPct * 64)

    m = Sequential()

    layer = 0 

    while neuronConunt>1 and layer<l:

        if layer==0:
            m.add(Conv2D(neuronConunt, (3,3), activation = 'relu', input_shape=(28,28,1)))
        else:
            m.add(Conv2D(neuronConunt, (3,3), activation = 'relu'))

        layer += 1

        m.add(MaxPooling2D(2,2))
        neuronConunt = int(neuronConunt * 2)

    m.add(Flatten())
    m.add(Dense(neuronConunt, activation = 'relu'))
    m.add(Dropout(dropout))
    m.add(Dense(10, activation = 'softmax'))


    return m

In [10]:
model = generate_model(0.2, 0.7166975503570835, 0.54)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 45)        450       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 45)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 90)        36540     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 90)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2250)              0         
_________________________________________________________________
dense (Dense)                (None, 180)               405180    
_________________________________________________________________
dropout (Dropout)            (None, 180)               0

In [11]:
def evaluate_network(dropout, lr, neuronPct, l_num):#,neuronShrink):
    
    rand = list(np.random.randint(1,1000,3))
    
    mean_benchmark = []
    #epochs_nedded = []
    num = 0
    
    for num_ix, rand_num in enumerate(rand):
        
        x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=rand_num)
        
        y_train = to_categorical(y_train)
        y_val = to_categorical(y_val)
        
        model = generate_model(dropout, neuronPct, l_num)
        opt = keras.optimizers.Adam(lr=lr)

        model.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])
        
        monitor = EarlyStopping(monitor='val_loss',  
        patience=3, verbose=0, mode='auto', restore_best_weights=True)
        
        model.fit(x_train,y_train,validation_data=(x_val,y_val),
                  callbacks=[monitor],verbose=0,epochs=10, batch_size = 64)
        #epochs = monitor.stopped_epoch
        #epochs_needed.append(epochs)
        
        _,acc = model.evaluate(X_test, y_test, verbose = 0)
        
        mean_benchmark.append(acc)
        m1 = statistics.mean(mean_benchmark)
        #m2 = statistics.mean(epochs_needed)
        #mdev = statistics.pstdev(mean_benchmark)

        #time_took = time.time() - start_time
        
        del model
        K.clear_session()

    return (m1)

In [12]:
pbounds = {'dropout':(0.0,0.49),
          'lr':(1e-4, 1e-2),
          'neuronPct': (0.1, 1), 
          'l_num':(0,1)}

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1)

optimizer.maximize(init_points=10, n_iter=3)
print(optimizer.max)

|   iter    |  target   |  dropout  |   l_num   |    lr     | neuronPct |
-------------------------------------------------------------------------
|  1        |  0.9792   |  0.2043   |  0.7203   |  0.000101 |  0.3721   |
|  2        |  0.9841   |  0.07191  |  0.09234  |  0.001944 |  0.411    |
|  3        |  0.9901   |  0.1944   |  0.5388   |  0.00425  |  0.7167   |
|  4        |  0.986    |  0.1002   |  0.8781   |  0.000371 |  0.7034   |
|  5        |  0.9896   |  0.2045   |  0.5587   |  0.00149  |  0.2783   |
|  6        |  0.9843   |  0.3924   |  0.9683   |  0.003203 |  0.7231   |


/home/jovyan/.local/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


KeyboardInterrupt: 